下記はリアルタイムでスクレイピングし前日のランキングを表示させるコードです
余裕がありストリームリット上で開発者画面を作る際に実装すると思います
その時は関数化したりもっと加工する必要がありますね
・・・すみません💦余裕なくてあんまりコメントアウトなどして解説いれきれてないです

In [1]:
"""
このスクリプトは、Project Gutenbergの人気書籍ランキングページから
書籍名、URL、読みやすさスコアを取得し、データフレームにまとめるものです。
"""

# 必要なライブラリをインポート
import requests  # HTTPリクエストを送信するためのライブラリ
from bs4 import BeautifulSoup  # HTMLを解析するためのライブラリ
import pandas as pd  # データ分析・操作のためのライブラリ
import time  # 時間関連の処理を行うためのライブラリ


# スクレイピングするURLを指定
#url = input("スクレイピングするURLを入力してください: ")  # ユーザーからURLを入力してもらう場合はこちらを使用
# 今回はデフォルトのURLを記載
url = 'https://www.gutenberg.org/browse/scores/top'  # Project Gutenbergの人気書籍ランキングページ

# ウェブページを取得
response = requests.get(url)  # 指定URLにHTTP GETリクエストを送信

# ステータスコードを確認（200は成功を意味する）
if response.status_code == 200:
    # HTMLを解析してBeautifulSoupオブジェクトを作成
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # タイトルを取得（h2タグの内容）
    top100_books = soup.select_one('h2').text  # CSSセレクタでh2タグを選択
    print(f"ページタイトル: {top100_books}")
    
    # 書籍タイトルとURLを格納するための空のリストを作成
    book_titles = []  # 書籍名を格納するリスト
    book_urls = []    # 書籍URLを格納するリスト
    reading_ease_scores = []  # 読みやすさスコアを格納するリスト

    # 説明用のヘッダー表示
    print('------------------------')
    print('昨日読まれた上位10冊は下記となります')
    print('------------------------')

    # トップ100の書籍名をCSSセレクタで取得（olタグ内のliタグ内のaタグ）
    books = soup.select('ol li a')  # リスト内のリンクをすべて取得
    for i, book in enumerate(books[:10], start=1):  # 上位10冊のみを処理、インデックスは1から開始
        # 書籍タイトルを取得して保存
        book_title = book.text.strip()  # テキストを抽出し、余分な空白を削除
        book_titles.append(book_title)  # リストに書籍タイトルを追加
        print(f"書籍名 {i}: {book_title}")  # 書籍名とランキングを表示

        # 書籍の詳細ページURLを生成して保存
        book_url = 'https://www.gutenberg.org' + book['href']  # 相対URLを絶対URLに変換
        print(book_url)  # 生成したURLを表示
        book_urls.append(book_url)  # リストにURLを追加

        #書籍の情報を更に取得するために該当のURLにアクセス
        #ここのコードかいてから一気に遅くなるので書籍情報とか不要ならなくてもいいかも
        book_response = requests.get(book_url)  # 書籍詳細ページにHTTP GETリクエストを送信
        soup_book = BeautifulSoup(book_response.text, 'html.parser')
        # テーブルの全ての行を取得
        note_rows = soup_book.select('table.bibrec tr')
        # 各行をループで処理して読みやすさスコアを探す
        for row in note_rows:
            # その行の中のth要素（見出し）を取得
            th_element = row.select_one('th')
            # th要素が存在し、その内容が'Note'である行を見つける
            if th_element and th_element.text.strip() == 'Note':
                # その行のtd要素（データ）を取得
                td_element = row.select_one('td')
                # td要素が存在し、その中に'Reading ease score'という文字列が含まれていたら
                if td_element and 'Reading ease score' in td_element.text:
                    # その内容を読みやすさスコアとして保存
                    reading_ease_score = td_element.text.strip().split(':')[1].split(' (')[0] #スコアの数字部分のみを取得
        print(f"読みやすさスコア: {reading_ease_score}")  # 読みやすさスコアを表示
        reading_ease_scores.append(reading_ease_score)  # リストにスコアを追加
        time.sleep(1)  # 1秒待機（サーバーへの負荷を軽減するため）

else:
    # ページ取得に失敗した場合のエラーメッセージ
    print(f"ページの取得に失敗しました。ステータスコード: {response.status_code}")

#print("スクレイピングが完了しました。")  # 処理完了のメッセージ

# 先にランキングのカラムを作成してから書籍名を追加する
# まずはランキングだけのデータフレームを作成
df_books = pd.DataFrame({'ランキング': range(1, len(book_titles) + 1)})  # 1から書籍数までの連番を生成

# 書籍名のカラムを追加
df_books['書籍名'] = book_titles  # 取得した書籍タイトルをデータフレームに追加

# 書籍URLのカラムを追加
df_books['書籍URL'] = book_urls  # 取得した書籍URLをデータフレームに追加

# 読みやすさスコアのカラムを追加
df_books['読みやすさスコア'] = reading_ease_scores  # 取得した読みやすさスコアをデータフレームに追加

# 結果を表示
print('------------------------')
print('データーフレームの作成が完了しました')
print('------------------------')
df_books  # データフレームを表示したい場合はコメントを外す

ページタイトル: Top 100 EBooks yesterday
------------------------
昨日読まれた上位10冊は下記となります
------------------------
書籍名 1: Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley (4538)
https://www.gutenberg.org/ebooks/84
読みやすさスコア:  65.0
書籍名 2: Moby Dick; Or, The Whale by Herman Melville (3202)
https://www.gutenberg.org/ebooks/2701
読みやすさスコア:  73.3
書籍名 3: Alice's Adventures in Wonderland by Lewis Carroll (2157)
https://www.gutenberg.org/ebooks/11
読みやすさスコア:  87.5
書籍名 4: Pride and Prejudice by Jane Austen (2114)
https://www.gutenberg.org/ebooks/1342
読みやすさスコア:  69.2
書籍名 5: Romeo and Juliet by William Shakespeare (1899)
https://www.gutenberg.org/ebooks/1513
読みやすさスコア:  92.6
書籍名 6: A Doll's House : a play by Henrik Ibsen (1891)
https://www.gutenberg.org/ebooks/2542
読みやすさスコア:  86.6
書籍名 7: The Great Gatsby by F. Scott  Fitzgerald (1870)
https://www.gutenberg.org/ebooks/64317
読みやすさスコア:  81.6
書籍名 8: The Importance of Being Earnest: A Trivial Comedy for Serious People by Oscar Wilde (1703)
http

,ランキング,書籍名,書籍URL,読みやすさスコア
0,1,"Frankenstein; Or, The Modern Prometheus by Mar...",https://www.gutenberg.org/ebooks/84,65.0
1,2,"Moby Dick; Or, The Whale by Herman Melville (3...",https://www.gutenberg.org/ebooks/2701,73.3
2,3,Alice's Adventures in Wonderland by Lewis Carr...,https://www.gutenberg.org/ebooks/11,87.5
3,4,Pride and Prejudice by Jane Austen (2114),https://www.gutenberg.org/ebooks/1342,69.2
4,5,Romeo and Juliet by William Shakespeare (1899),https://www.gutenberg.org/ebooks/1513,92.6
5,6,A Doll's House : a play by Henrik Ibsen (1891),https://www.gutenberg.org/ebooks/2542,86.6
6,7,The Great Gatsby by F. Scott Fitzgerald (1870),https://www.gutenberg.org/ebooks/64317,81.6
7,8,The Importance of Being Earnest: A Trivial Com...,https://www.gutenberg.org/ebooks/844,77.9
8,9,The Picture of Dorian Gray by Oscar Wilde (1404),https://www.gutenberg.org/ebooks/174,84.0
9,10,The Complete Works of William Shakespeare by W...,https://www.gutenberg.org/ebooks/100,83.0


下記からは具体的な書籍をスクレイピングするコードです
今回は直接CSVと画像データをデプロイ先に格納するのでストリームリット上には反映しないかも
・・・すみません💦余裕なくてあんまりコメントアウトなどして解説いれきれてないです

In [ ]:
"""
不思議の国のアリスでスクレイピング
まずは全体をスクレイピング
streamlitで実行する場合は不要なコメントアウトやprint文を削除
"""
import os
from dotenv import load_dotenv
load_dotenv()  # .envファイルを読み込む
import requests
from bs4 import BeautifulSoup
import pandas as pd
#import time 今回はスクレイピングが1度きりなので不要
import re
import sys
import deepl

url = 'https://www.gutenberg.org/cache/epub/11/pg11-images.html'
#response = requests.get(url)

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print('======================')
    print(f"ページを取得しました")
    print('======================')
except requests.exceptions.RequestException as e:
    print(f"{url}の取得に失敗しました: {e}")
    print('コードの実行を終了します')
    sys.exit(1)  # エラーが発生した場合はプログラムを終了


"""
1.書籍のタイトル名
タイトルを取得してリストに格納
"""
book_title = soup.select_one('h1').text  # CSSセレクタでh1タグを選択
print('======================')
print(f"書籍名: {book_title}")
print('======================')




"""
2.チャプター毎のタイトル名
チャプターのタイトルを取得してリストに格納
"""
# print('チャプタータイトルをスクレイピングします')

chapter_titles = soup.select('div.chapter h2')
chapter_list = []

for i, chapter_title in enumerate(chapter_titles, start=1):
    #改行を含む文字列なので改行を削除
    title = chapter_title.text.replace('\n', '').strip().split('\r')[1]
    # print(f"{i}章：{title}")
    #titleをリストに追加
    chapter_list.append(title)
# print('======================')
# print(f"合計{len(chapter_titles)}個リストに格納しました")
# print('======================')

print('======================')
print(f'章毎のタイトルリストを表示します: {chapter_list}')
print('======================')


"""
3.本文（チャプターの内容）
1行ずつ<p></p>になっているので結合していく必要がある
"""

# print('本文をスクレイピングします')


# 空のリストを作成
sentences_list = []

chapter_divs = soup.select('div.chapter')  # 各チャプターを取得

# 各チャプターを処理
for i, chapter in enumerate(chapter_divs, start=1):
    # print(f'{i}章を処理します')

    # このチャプターの全段落を取得
    chapter_sentences = chapter.select('p')

    # 段落を結合するための空の文字列
    chapter_text = ''

    # 各段落を処理
    for p in chapter_sentences:
        # 改行などを削除してきれいにする
        clean_text = p.text.replace('\r', '').replace('\n', '').replace('\xa0', '')#正規表現で特殊文字を削除
        # 文字列に追加
        chapter_text += clean_text
        #さらに正規表現で特殊文字を削除
        # アスタリスクの繰り返しパターンを削除する例
        chapter_text = re.sub(r'\*\s*\*\s*\*\s*\*', ' ', chapter_text)
    # 結合した章のテキストをリストに追加
    sentences_list.append(chapter_text)

    #print(f'第{i}章の処理完了！({len(chapter_sentences)}個の段落を結合)')

# print('======================')
# print(f'全{len(sentences_list)}章の処理が終了しました')
# print('======================')


"""
本文を章単位で表示
"""
print('======================')
print(f'本文のリストを表示します: {sentences_list}')
print('======================')


df_Alice = pd.DataFrame({'章タイトル':chapter_list, '本文': sentences_list})

"""
画像を取得するコード
"""

from urllib.parse import urljoin

_response = requests.get(url)
_soup = BeautifulSoup(_response.content, 'html.parser')

book_img_tag = _soup.select_one('img')
book_img_url = urljoin(url, book_img_tag['src'])



"""
画像を保存するコード
ストリームリット上で画像を取得したい場合に有効化
今回は直接フォルダ上に画像を入れるので大丈夫
"""

# book_img = requests.get(book_img_url)
# book_img_title = 'Alice'

# with open(f'{book_img_title}_img.jpg', 'wb') as f:
#     f.write(book_img.content)
#     print(f'{book_img_title}_img.jpgを保存しました')




""""
要約の取得
"""
#DeepLのAPIを使用するのでAPIキーが必要になります

auth_key = os.getenv('DeepL_API_KEY')  # 環境変数から認証キーを取得
translator = deepl.Translator(auth_key)


url = 'https://www.gutenberg.org/ebooks/11'

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # print('======================')
    # print(f"ページを取得しました")
    # print('======================')
except requests.exceptions.RequestException as e:
    # print(f"{url}の取得に失敗しました: {e}")
    # print('コードの実行を終了します')
    sys.exit(1)  # エラーが発生した場合はプログラムを終了

summary = soup.select_one('span.readmore-container')
summary_text = summary.text.replace('\n', '').split('(')[0]

result = translator.translate_text(summary_text, target_lang='JA')
summary_text_JA = result.text.split('。')[1]
print('======================')
print(f"要約: {summary_text_JA}")
print('======================')

df_Alice = pd.DataFrame({'書籍タイトル': book_title,'要約_英': summary_text, '要約_和': summary_text_JA, '画像URL': book_img_url,'章タイトル': chapter_list, '本文': sentences_list})
df_Alice

ページを取得しました
書籍名: Alice’s Adventures in Wonderland
章毎のタイトルリストを表示します: ['Down the Rabbit-Hole', 'The Pool of Tears', 'A Caucus-Race and a Long Tale', 'The Rabbit Sends in a Little Bill', 'Advice from a Caterpillar', 'Pig and Pepper', 'A Mad Tea-Party', 'The Queen’s Croquet-Ground', 'The Mock Turtle’s Story', 'The Lobster Quadrille', 'Who Stole the Tarts?', 'Alice’s Evidence']
本文のリストを表示します: ['Alice was beginning to get very tired of sitting by her sister on the bank, andof having nothing to do: once or twice she had peeped into the book her sisterwas reading, but it had no pictures or conversations in it, “and what isthe use of a book,” thought Alice “without pictures orconversations?”So she was considering in her own mind (as well as she could, for the hot daymade her feel very sleepy and stupid), whether the pleasure of making adaisy-chain would be worth the trouble of getting up and picking the daisies,when suddenly a White Rabbit with pink eyes ran close by her.There was nothing so very

,書籍タイトル,要約_英,要約_和,画像URL,章タイトル,本文
0,Alice’s Adventures in Wonderland,"""Alice's Adventures in Wonderland"" by Lewis Ca...",この物語は、アリスという名の少女が、川辺で座っているときに退屈で眠くなり、白ウサギに出会って...,https://www.gutenberg.org/cache/epub/11/images...,Down the Rabbit-Hole,Alice was beginning to get very tired of sitti...
1,Alice’s Adventures in Wonderland,"""Alice's Adventures in Wonderland"" by Lewis Ca...",この物語は、アリスという名の少女が、川辺で座っているときに退屈で眠くなり、白ウサギに出会って...,https://www.gutenberg.org/cache/epub/11/images...,The Pool of Tears,“Curiouser and curiouser!” cried Alice (she wa...
2,Alice’s Adventures in Wonderland,"""Alice's Adventures in Wonderland"" by Lewis Ca...",この物語は、アリスという名の少女が、川辺で座っているときに退屈で眠くなり、白ウサギに出会って...,https://www.gutenberg.org/cache/epub/11/images...,A Caucus-Race and a Long Tale,They were indeed a queer-looking party that as...
3,Alice’s Adventures in Wonderland,"""Alice's Adventures in Wonderland"" by Lewis Ca...",この物語は、アリスという名の少女が、川辺で座っているときに退屈で眠くなり、白ウサギに出会って...,https://www.gutenberg.org/cache/epub/11/images...,The Rabbit Sends in a Little Bill,"It was the White Rabbit, trotting slowly back ..."
4,Alice’s Adventures in Wonderland,"""Alice's Adventures in Wonderland"" by Lewis Ca...",この物語は、アリスという名の少女が、川辺で座っているときに退屈で眠くなり、白ウサギに出会って...,https://www.gutenberg.org/cache/epub/11/images...,Advice from a Caterpillar,The Caterpillar and Alice looked at each other...
5,Alice’s Adventures in Wonderland,"""Alice's Adventures in Wonderland"" by Lewis Ca...",この物語は、アリスという名の少女が、川辺で座っているときに退屈で眠くなり、白ウサギに出会って...,https://www.gutenberg.org/cache/epub/11/images...,Pig and Pepper,For a minute or two she stood looking at the h...
6,Alice’s Adventures in Wonderland,"""Alice's Adventures in Wonderland"" by Lewis Ca...",この物語は、アリスという名の少女が、川辺で座っているときに退屈で眠くなり、白ウサギに出会って...,https://www.gutenberg.org/cache/epub/11/images...,A Mad Tea-Party,There was a table set out under a tree in fron...
7,Alice’s Adventures in Wonderland,"""Alice's Adventures in Wonderland"" by Lewis Ca...",この物語は、アリスという名の少女が、川辺で座っているときに退屈で眠くなり、白ウサギに出会って...,https://www.gutenberg.org/cache/epub/11/images...,The Queen’s Croquet-Ground,A large rose-tree stood near the entrance of t...
8,Alice’s Adventures in Wonderland,"""Alice's Adventures in Wonderland"" by Lewis Ca...",この物語は、アリスという名の少女が、川辺で座っているときに退屈で眠くなり、白ウサギに出会って...,https://www.gutenberg.org/cache/epub/11/images...,The Mock Turtle’s Story,“You can’t think how glad I am to see you agai...
9,Alice’s Adventures in Wonderland,"""Alice's Adventures in Wonderland"" by Lewis Ca...",この物語は、アリスという名の少女が、川辺で座っているときに退屈で眠くなり、白ウサギに出会って...,https://www.gutenberg.org/cache/epub/11/images...,The Lobster Quadrille,"The Mock Turtle sighed deeply, and drew the ba..."


In [ ]:
"""
不思議の国のアリスでスクレイピング
まずは全体をスクレイピング
streamlitで実行する場合は不要なコメントアウトやprint文を削除
"""
import os
from dotenv import load_dotenv
load_dotenv()  # .envファイルを読み込む
import requests
from bs4 import BeautifulSoup
import pandas as pd
#import time 今回はスクレイピングが1度きりなので不要
import re
import sys
import deepl

url = 'https://www.gutenberg.org/cache/epub/11/pg11-images.html'
#response = requests.get(url)

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print('======================')
    print(f"ページを取得しました")
    print('======================')
except requests.exceptions.RequestException as e:
    print(f"{url}の取得に失敗しました: {e}")
    print('コードの実行を終了します')
    sys.exit(1)  # エラーが発生した場合はプログラムを終了


"""
1.書籍のタイトル名
タイトルを取得してリストに格納
"""
book_title = soup.select_one('h1').text  # CSSセレクタでh1タグを選択
print('======================')
print(f"書籍名: {book_title}")
print('======================')




"""
2.チャプター毎のタイトル名
チャプターのタイトルを取得してリストに格納
"""
# print('チャプタータイトルをスクレイピングします')

chapter_titles = soup.select('div.chapter h2')
chapter_list = []

for i, chapter_title in enumerate(chapter_titles, start=1):
    #改行を含む文字列なので改行を削除
    title = chapter_title.text.replace('\n', '').strip().split('\r')[1]
    # print(f"{i}章：{title}")
    #titleをリストに追加
    chapter_list.append(title)
# print('======================')
# print(f"合計{len(chapter_titles)}個リストに格納しました")
# print('======================')

print('======================')
print(f'章毎のタイトルリストを表示します: {chapter_list}')
print('======================')


"""
3.本文（チャプターの内容）
1行ずつ<p></p>になっているので結合していく必要がある
"""

# print('本文をスクレイピングします')


# 空のリストを作成
sentences_list = []

chapter_divs = soup.select('div.chapter')  # 各チャプターを取得

# 各チャプターを処理
for i, chapter in enumerate(chapter_divs, start=1):
    # print(f'{i}章を処理します')

    # このチャプターの全段落を取得
    chapter_sentences = chapter.select('p')

    # 段落を結合するための空の文字列
    chapter_text = ''

    # 各段落を処理
    for p in chapter_sentences:
        # 改行などを削除してきれいにする
        clean_text = p.text.replace('\r', '').replace('\n', '').replace('\xa0', '')#正規表現で特殊文字を削除
        # 文字列に追加
        chapter_text += clean_text
        #さらに正規表現で特殊文字を削除
        # アスタリスクの繰り返しパターンを削除する例
        chapter_text = re.sub(r'\*\s*\*\s*\*\s*\*', ' ', chapter_text)
    # 結合した章のテキストをリストに追加
    sentences_list.append(chapter_text)

    #print(f'第{i}章の処理完了！({len(chapter_sentences)}個の段落を結合)')

# print('======================')
# print(f'全{len(sentences_list)}章の処理が終了しました')
# print('======================')


"""
本文を章単位で表示
"""
print('======================')
print(f'本文のリストを表示します: {sentences_list}')
print('======================')


df_Alice = pd.DataFrame({'章タイトル':chapter_list, '本文': sentences_list})

"""
画像を取得するコード
"""

from urllib.parse import urljoin

_response = requests.get(url)
_soup = BeautifulSoup(_response.content, 'html.parser')

book_img_tag = _soup.select_one('img')
book_img_url = urljoin(url, book_img_tag['src'])



"""
画像を保存するコード
ストリームリット上で画像を取得したい場合に有効化
今回は直接フォルダ上に画像を入れるので大丈夫
"""

# book_img = requests.get(book_img_url)
# book_img_title = 'Alice'

# with open(f'{book_img_title}_img.jpg', 'wb') as f:
#     f.write(book_img.content)
#     print(f'{book_img_title}_img.jpgを保存しました')




""""
要約の取得
"""
#DeepLのAPIを使用するのでAPIキーが必要になります

auth_key = os.getenv('DeepL_API_KEY')  # 環境変数から認証キーを取得
translator = deepl.Translator(auth_key)


url = 'https://www.gutenberg.org/ebooks/11'

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # print('======================')
    # print(f"ページを取得しました")
    # print('======================')
except requests.exceptions.RequestException as e:
    # print(f"{url}の取得に失敗しました: {e}")
    # print('コードの実行を終了します')
    sys.exit(1)  # エラーが発生した場合はプログラムを終了

summary = soup.select_one('span.readmore-container')
summary_text = summary.text.replace('\n', '').split('(')[0]

result = translator.translate_text(summary_text, target_lang='JA')
summary_text_JA = result.text.split('。')[1]
print('======================')
print(f"要約: {summary_text_JA}")
print('======================')

df_Alice = pd.DataFrame({'書籍タイトル': book_title,'要約_英': summary_text, '要約_和': summary_text_JA, '画像URL': book_img_url,'章タイトル': chapter_list, '本文': sentences_list})
df_Alice

In [ ]:
"""
オズの魔法使いでスクレイピング
まずは全体をスクレイピング
streamlitで実行する場合は不要なコメントアウトやprint文を削除
"""
import os
from dotenv import load_dotenv
load_dotenv()  # .envファイルを読み込む
import requests
from bs4 import BeautifulSoup
import pandas as pd
#import time 今回はスクレイピングが1度きりなので不要
import re
import sys
import deepl

url = 'https://www.gutenberg.org/cache/epub/55/pg55-images.html'
#response = requests.get(url)

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print('======================')
    print(f"ページを取得しました")
    print('======================')
except requests.exceptions.RequestException as e:
    print(f"{url}の取得に失敗しました: {e}")
    print('コードの実行を終了します')
    sys.exit(1)  # エラーが発生した場合はプログラムを終了


"""
1.書籍のタイトル名
タイトルを取得してリストに格納
"""
book_title = soup.select_one('h1').text  # CSSセレクタでh1タグを選択
print('======================')
print(f"書籍名: {book_title}")
print('======================')




"""
2.チャプター毎のタイトル名
チャプターのタイトルを取得してリストに格納
"""
# print('チャプタータイトルをスクレイピングします')

chapter_titles = soup.select('div.chapter h2')
chapter_titles
chapter_list = []

for i, chapter_title in enumerate(chapter_titles[2:]):
    chapter_title = chapter_title.text.split('\n')[1]
    print(f"第{i+1}章: {chapter_title}")
    chapter_list.append(chapter_title)
print('======================')
print(f"合計{len(chapter_titles)}個リストに格納しました")
print('======================')

# print('======================')
# print(f'章毎のタイトルリストを表示します: {chapter_list}')
# print('======================')


"""
3.本文（チャプターの内容）
1行ずつ<p></p>になっているので結合していく必要がある
"""

# print('本文をスクレイピングします')
# 空のリストを作成
sentences_list = []

chapter_divs = soup.select('div.chapter')  # 各チャプターを取得

# 各チャプターを処理
for i, chapter in enumerate(chapter_divs[2:], start=1):#イントロダクションと空白のチャプターがあるので2からスタート
    # print(f'{i}章を処理します')

    # このチャプターの全段落を取得
    chapter_sentences = chapter.select('p')

    # 段落を結合するための空の文字列
    chapter_text = ''

    # 各段落を処理
    for p in chapter_sentences:
        # 改行などを削除してきれいにする
        clean_text = p.text.replace('\r', '').replace('\n', '').replace('\xa0', '')#正規表現で特殊文字を削除
        # 文字列に追加
        chapter_text += clean_text
        #さらに正規表現で特殊文字を削除
        # アスタリスクの繰り返しパターンを削除する例
        # chapter_text = re.sub(r'\*\s*\*\s*\*\s*\*', ' ', chapter_text)
    # 結合した章のテキストをリストに追加
    sentences_list.append(chapter_text)

    print(f'第{i}章の処理完了！({len(chapter_sentences)}個の段落を結合)')

print('======================')
print(f'全{len(sentences_list)}章の処理が終了しました')
print('======================')

"""
画像を取得するコード
"""

from urllib.parse import urljoin

_response = requests.get(url)
_soup = BeautifulSoup(_response.content, 'html.parser')

book_img_tag = _soup.select_one('img')
book_img_url = urljoin(url, book_img_tag['src'])



"""
画像を保存するコード
ストリームリット上で画像を取得したい場合に有効化
今回は直接フォルダ上に画像を入れるので大丈夫
"""

# book_img = requests.get(book_img_url)
# book_img_title = 'Oz'

# with open(f'{book_img_title}_img.jpg', 'wb') as f:
#     f.write(book_img.content)
#     print(f'{book_img_title}_img.jpgを保存しました')

""""
要約の取得
"""


auth_key = os.getenv('DeepL_API_KEY')  # 環境変数から認証キーを取得
translator = deepl.Translator(auth_key)


url = 'https://www.gutenberg.org/ebooks/55'

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # print('======================')
    # print(f"ページを取得しました")
    # print('======================')
except requests.exceptions.RequestException as e:
    # print(f"{url}の取得に失敗しました: {e}")
    # print('コードの実行を終了します')
    sys.exit(1)  # エラーが発生した場合はプログラムを終了

summary = soup.select_one('span.readmore-container')
summary_text = summary.text.replace('\n', '').split('(')[0]

result = translator.translate_text(summary_text, target_lang='JA')
summary_text_JA = result.text.split('。')[1]
print('======================')
print(f"要約: {summary_text_JA}")
print('======================')

df_Oz = pd.DataFrame({'書籍タイトル': book_title,'要約_英': summary_text, '要約_和': summary_text_JA, '画像URL': book_img_url,'章タイトル': chapter_list, '本文': sentences_list})

In [ ]:
"""
ピーターパンでスクレイピング
まずは全体をスクレイピング
streamlitで実行する場合は不要なコメントアウトやprint文を削除
"""
import os
from dotenv import load_dotenv
load_dotenv()  # .envファイルを読み込む
import requests
from bs4 import BeautifulSoup
import pandas as pd
#import time 今回はスクレイピングが1度きりなので不要
import re
import sys
import deepl

url = 'https://www.gutenberg.org/cache/epub/16/pg16-images.html'
#response = requests.get(url)

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print('======================')
    print(f"ページを取得しました")
    print('======================')
except requests.exceptions.RequestException as e:
    print(f"{url}の取得に失敗しました: {e}")
    print('コードの実行を終了します')
    sys.exit(1)  # エラーが発生した場合はプログラムを終了


"""
1.書籍のタイトル名
タイトルを取得してリストに格納
"""
book_title = soup.select_one('h1').text  # CSSセレクタでh1タグを選択
print('======================')
print(f"書籍名: {book_title}")
print('======================')




"""
2.チャプター毎のタイトル名
チャプターのタイトルを取得してリストに格納
"""
# print('チャプタータイトルをスクレイピングします')

chapter_titles = soup.select('div.chapter h2')
chapter_titles
chapter_list = []

for i, chapter_title in enumerate(chapter_titles):
    chapter_title = chapter_title.text.split('\n')[1]
    print(f"第{i+1}章: {chapter_title}")
    chapter_list.append(chapter_title)
print('======================')
print(f"合計{len(chapter_titles)}個リストに格納しました")
print('======================')

# print('======================')
# print(f'章毎のタイトルリストを表示します: {chapter_list}')
# print('======================')


"""
3.本文（チャプターの内容）
1行ずつ<p></p>になっているので結合していく必要がある
"""

# print('本文をスクレイピングします')
# 空のリストを作成
sentences_list = []

chapter_divs = soup.select('div.chapter')  # 各チャプターを取得

# 各チャプターを処理
for i, chapter in enumerate(chapter_divs, start=1):#イントロダクションと空白のチャプターがあるので2からスタート
    # print(f'{i}章を処理します')

    # このチャプターの全段落を取得
    chapter_sentences = chapter.select('p')

    # 段落を結合するための空の文字列
    chapter_text = ''

    # 各段落を処理
    for p in chapter_sentences:
        # 改行などを削除してきれいにする
        clean_text = p.text.replace('\r', '').replace('\n', '').replace('\xa0', '')#正規表現で特殊文字を削除
        # 文字列に追加
        chapter_text += clean_text
        #さらに正規表現で特殊文字を削除
        # アスタリスクの繰り返しパターンを削除する例
        # chapter_text = re.sub(r'\*\s*\*\s*\*\s*\*', ' ', chapter_text)
    # 結合した章のテキストをリストに追加
    sentences_list.append(chapter_text)

    print(f'第{i}章の処理完了！({len(chapter_sentences)}個の段落を結合)')

print('======================')
print(f'全{len(sentences_list)}章の処理が終了しました')
print('======================')

"""
画像を取得するコード
ここは今までとはコードが違うので注意
"""

from urllib.parse import urljoin

url = 'https://www.gutenberg.org/ebooks/16'

_response = requests.get(url)
_soup = BeautifulSoup(_response.content, 'html.parser')

book_img_tag = _soup.select_one('img.cover-art')
book_img_url = urljoin(url, book_img_tag['src'])



"""
画像を保存するコード
ストリームリット上で画像を取得したい場合に有効化
今回は直接フォルダ上に画像を入れるので大丈夫
"""

# book_img = requests.get(book_img_url)
# book_img_title = 'PeterPan'

# with open(f'{book_img_title}_img.jpg', 'wb') as f:
#     f.write(book_img.content)
#     print(f'{book_img_title}_img.jpgを保存しました')

""""
要約の取得
"""


auth_key = os.getenv('DeepL_API_KEY')  # 環境変数から認証キーを取得
translator = deepl.Translator(auth_key)


url = 'https://www.gutenberg.org/ebooks/16'

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # print('======================')
    # print(f"ページを取得しました")
    # print('======================')
except requests.exceptions.RequestException as e:
    # print(f"{url}の取得に失敗しました: {e}")
    # print('コードの実行を終了します')
    sys.exit(1)  # エラーが発生した場合はプログラムを終了

summary = soup.select_one('span.readmore-container')
summary_text = summary.text.replace('\n', '').split('(')[0]

result = translator.translate_text(summary_text, target_lang='JA')
summary_text_JA = result.text.split('。')[1]
print('======================')
print(f"要約: {summary_text_JA}")
print('======================')

df_PeterPan = pd.DataFrame({'書籍タイトル': book_title,'要約_英': summary_text, '要約_和': summary_text_JA, '画像URL': book_img_url,'章タイトル': chapter_list, '本文': sentences_list})

In [ ]:
"""
秘密の花園でスクレイピング
まずは全体をスクレイピング
streamlitで実行する場合は不要なコメントアウトやprint文を削除
"""
import os
from dotenv import load_dotenv
load_dotenv()  # .envファイルを読み込む
import requests
from bs4 import BeautifulSoup
import pandas as pd
#import time 今回はスクレイピングが1度きりなので不要
import re
import sys
import deepl

url = 'https://www.gutenberg.org/cache/epub/17396/pg17396-images.html'
#response = requests.get(url)

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    print('======================')
    print(f"ページを取得しました")
    print('======================')
except requests.exceptions.RequestException as e:
    print(f"{url}の取得に失敗しました: {e}")
    print('コードの実行を終了します')
    sys.exit(1)  # エラーが発生した場合はプログラムを終了


"""
1.書籍のタイトル名
タイトルを取得してリストに格納
"""
book_title = soup.select('h1')  # CSSセレクタでh1タグを選択
#リスト型に別れてしまったので結合
empty_title = ''
for i in  book_title:
    book_title = i.text
    empty_title += book_title
book_title = empty_title

print('======================')
print(f"書籍名: {book_title}")
print('======================')




"""
2.チャプター毎のタイトル名
チャプターのタイトルを取得してリストに格納
"""
# print('チャプタータイトルをスクレイピングします')

chapter_titles = soup.select('h3')
chapter_titles
chapter_list = []

for i, chapter_title in enumerate(chapter_titles[1:-1], start=1):##最初と最後のタイトルは不要なのでスライス
    chapter_title = chapter_title.text
    print(f"第{i}章: {chapter_title}")
    chapter_list.append(chapter_title)
print('======================')
print(f"合計{len(chapter_titles)}個リストに格納しました")
print('======================')

# print('======================')
# print(f'章毎のタイトルリストを表示します: {chapter_list}')
# print('======================')

"""
3.本文（チャプターの内容）
1行ずつ<p></p>になっているので結合していく必要がある
今までと違いdivでchapterが別れてないのうまいこと調整しないといけない
"""
sentences_list = []    # 章ごとの本文

h3_tags = soup.find_all('h3')

for h3 in h3_tags:
    chapter_text = ''  # この章の本文
    # h3～次のh3の直前までのpタグを集める
    sibling = h3.next_sibling
    while sibling:
        # タグだったら
        if getattr(sibling, 'name', None):
            if sibling.name == 'h2':
                break  # 次の章タイトルが来たら終了！
            if sibling.name == 'p':
                chapter_text += sibling.text.strip().replace('\r', '').replace('\n', '').replace('\\', '')
                chapter_text = re.sub(r'\[Pg \d+\]', '', chapter_text)
        sibling = sibling.next_sibling

    sentences_list.append(chapter_text)

# 章タイトルの最初と最後の要素は不要なので削除
sentences_list = sentences_list[1:-1]


print('======================')
print(f'全{len(sentences_list)}章の処理が終了しました')
print('======================')

"""
画像を取得するコード
"""

from urllib.parse import urljoin

_response = requests.get(url)
_soup = BeautifulSoup(_response.content, 'html.parser')

book_img_tag = _soup.select_one('img')
book_img_url = urljoin(url, book_img_tag['src'])



"""
画像を保存するコード
ストリームリット上で画像を取得したい場合に有効化
今回は直接フォルダ上に画像を入れるので大丈夫
"""

# book_img = requests.get(book_img_url)
# book_img_title = 'Garden'

# with open(f'{book_img_title}_img.jpg', 'wb') as f:
#     f.write(book_img.content)
#     print(f'{book_img_title}_img.jpgを保存しました')

""""
要約の取得
"""


auth_key = os.getenv('DeepL_API_KEY')  # 環境変数から認証キーを取得
translator = deepl.Translator(auth_key)


url = 'https://www.gutenberg.org/ebooks/17396'

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # print('======================')
    # print(f"ページを取得しました")
    # print('======================')
except requests.exceptions.RequestException as e:
    # print(f"{url}の取得に失敗しました: {e}")
    # print('コードの実行を終了します')
    sys.exit(1)  # エラーが発生した場合はプログラムを終了

summary = soup.select_one('span.readmore-container')
summary_text = summary.text.replace('\n', '').split('(')[0]
summary_text

result = translator.translate_text(summary_text, target_lang='JA')
summary_text_JA = result.text.split('。')[1]
print('======================')
print(f"要約: {summary_text_JA}")
print('======================')

df_garden = pd.DataFrame({'書籍タイトル': book_title,'要約_英': summary_text, '要約_和': summary_text_JA, '画像URL': book_img_url,'章タイトル': chapter_list, '本文': sentences_list})